#  PathFinder+: Advanced Career Engine Demo
This notebook demonstrates the **Responsibility Band** and **Multi-Signal Assessment** logic implemented in the PathFinder+ project.

### Core Concepts:
1. Responsibility Bands (0-4): Moving from titles to skill-depth.
2. Adaptive Scoring: Handling users with or without resumes.
3. Professional Top-Ups: Identifying the fastest way to the next level.

In [ ]:
import sys
from pathlib import Path
import json
import pandas as pd

# Set paths
project_root = Path("..")
sys.path.append(str(project_root / "scripts"))

from recommendation_engine import RecommendationEngine

engine = RecommendationEngine(
    jobs_path=project_root / "data/processed/all_jobs_master.csv",
    courses_path=project_root / "data/processed/all_courses_master.csv",
    esco_dir=project_root / "data/raw/esco"
)

# 1. AI Auto-Profiling (The Resume Signal)
The engine uses SBERT to estimate your Responsibility Band based on your resume text.

In [ ]:
resume_path = project_root / "resume_test/lorion.pdf" # Sample resume
profile = engine.auto_profile(resume_path)

print(f"Suggested Target: {profile['suggested_target']}")
print(f"Estimated Band: {profile['band']} (0=Intern, 4=Executive)")
print(f"Extracted Skills: {profile['extracted_skills'][:5]}...")

## 2. Band-Driven Career Progression
Instead of a static list, the progression now adapts to your current Band level.

In [ ]:
progression = engine.get_career_progression(
    current_role=profile['suggested_target'],
    current_band=profile['band'],
    user_skills=profile['extracted_skills']
)

print("### Vertical Promotion (Next Band)")
v_path = next(p for p in progression if p['type'] == 'Vertical (Promotion)')
print(f"Target Role: {v_path['role']}")
print(f"Advice: {v_path['advice']}")

print("\n### Horizontal Transition (Same Band)")
h_path = next(p for p in progression if p['type'] == 'Horizontal (Transition)')
print(f"Role: {h_path['role']}")

## 3. Professional Top-Ups
The system identifies the most efficient education path to reach the next Band.

In [ ]:
target_band = profile['band'] + 1
top_ups = engine.get_top_up_recommendations(
    current_band=profile['band'],
    target_band=target_band,
    segment="Professional"
)

print(f"To move to Band {target_band}, we recommend focus on:")
for t in top_ups:
    print(f"- {t}")

## 4. Multi-Signal Scoring (Adaptive Logic)
Demonstrating how the score changes if the user **skips the resume** and only does the assessment.

In [ ]:
target_skills = engine.get_skills_for_job("Senior Software Engineer")

score_with_resume = engine.calculate_skill_score(
    resume_skills=profile['extracted_skills'],
    assessment_data=[{'is_correct': True}, {'is_correct': False}],
    target_skills=target_skills
)

print(f"Skill Score WITH Resume: {score_with_resume}")

score_no_resume = engine.calculate_skill_score(
    resume_skills=[],
    assessment_data=[{'is_correct': True}, {'is_correct': False}],
    target_skills=target_skills
)

print(f"Skill Score WITHOUT Resume (Adaptive): {score_no_resume}")